In [146]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import Dataset

In [147]:
import pandas as pd
import numpy as np
import seaborn as sns

In [148]:
df=pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
df.head()

Next step is to understand the dataset

In [149]:
df.info()

In [150]:
df.columns

In [151]:
df.shape

In [152]:
df.sample(5)

In [153]:
df.isnull().sum()

In [154]:
df['FastingBS'].value_counts()

In [155]:
sns.countplot(x="FastingBS",data=df)

In [156]:
df['HeartDisease'].value_counts()

In [157]:
df['ChestPainType'].value_counts()

In [158]:
sns.countplot(x='ChestPainType',data=df)

In [159]:
df['RestingECG'].value_counts()

In [160]:
sns.countplot(x='RestingECG',data=df)

There are no null values present in this dataset. Lets check if there are any miscategeorized values.

Lets encode the required data

In [161]:
df.columns

In [162]:
df1=df
categorical_columns = ['Sex', 'ChestPainType', 
       'RestingECG',  'ExerciseAngina', 'ST_Slope',
      ]

for column in categorical_columns:
    cat = pd.get_dummies(df[column], prefix=column)

    df1 = pd.merge(
        left=df1,
        right=cat,
        left_index=True,
        right_index=True,
    )

    df1 = df1.drop(columns=column)



In [163]:
df1

In [164]:

import matplotlib.pyplot as plt
plt.figure(figsize = (16,5))
l=df.corr()
sns.heatmap(l,cmap="YlGnBu", annot=True)

In [165]:

plt.figure(figsize = (16,5))
l=df1.corr()
sns.heatmap(l,cmap="YlGnBu", annot=True)

**Here we have the important observation the correlation of all independent variables with respect to heart disease is better shown after encoding than before**

**We can say that every independent variable has some effect on dependent variable**

Feature Scaling

In [166]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y=df1['HeartDisease']
df2=df1.drop('HeartDisease',axis=1)
col=df2.columns
df2=scaler.fit_transform(df2)
df2

In [167]:
df2=pd.DataFrame(df2,columns=col)
df2

**Prediction part**

Our problem is of type classification .So we will first get training and testing data and will work with various models.

In [168]:
from sklearn.model_selection import train_test_split
X=df2

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [169]:
x_train.sample(5)

In [170]:
y_test.head()

In [171]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,roc_auc_score,classification_report,confusion_matrix
def metric(y_test,y_pred):
    h={}
    h['metrics']=classification_report(y_test,y_pred)
    h['accuracy']=accuracy_score(y_test,y_pred)
    h['confusion_matrix']=confusion_matrix(y_test,y_pred)
    return h

**1st model decisiion tree**

In [172]:
from sklearn.tree import DecisionTreeClassifier 
model1=DecisionTreeClassifier()
model1.fit(x_train,y_train)
y1_pred=model1.predict(x_test)

In [173]:
l=metric(y_test,y1_pred)
for i in l:
    print(l[i])

**knn**

In [174]:
from sklearn.neighbors import KNeighborsClassifier
model2=KNeighborsClassifier()
model2.fit(x_train,y_train)
y2_pred=model2.predict(x_test)

In [175]:
l=metric(y_test,y2_pred)
for i in l:
    print(l[i])

**lOGISTIC**

In [176]:
from sklearn.linear_model import LogisticRegression
model3=LogisticRegression()
model3.fit(x_train,y_train)
y3_pred=model3.predict(x_test)

In [177]:
l=metric(y_test,y3_pred)
for i in l:
    print(l[i])

**Random Forest**

In [178]:
from sklearn.ensemble import RandomForestClassifier
model4=RandomForestClassifier()
model4.fit(x_train,y_train)
y4_pred=model4.predict(x_test)

In [179]:
l=metric(y_test,y4_pred)
for i in l:
    print(l[i])

**Based on verification of all the models the data is best fit for knn,logistic with a good accuracy around 86%**